In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".13"

In [2]:
import pandas as pd
from brain import BrainModel
from submodels import factories
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
from random import shuffle

from lib.score import (
    fate_corr, score_both_size_norm, shrink_and_align_stats, score_stats_norm
)
from lib.preprocess import *
from lib.callback import (
    cell_number_callback, progenitor_number_callback, neuron_number_callback,
    TargetPopulation, TagNumberCallback,
)
from lib.sde.grn.grn4 import GRNMain4 as GRNMain
from lib.sde.mutate import mutate_grn4 as mutate_grn

from lib.ga.utils import weighted_selection_one, normalize_fitness_values
from jf.utils.export import Exporter
from jf.autocompute.jf import O, L
from itertools import product
import jf.models.stringmodel as sm
from lib.analyser import show_curve, show_curve_progenitor

In [3]:
HISTORY = defaultdict(dict)
HALL_OF_FAME = []

In [4]:
_count = -1
def provide_id():
    global _count
    _count += 1
    return _count

In [5]:
REF = O(
    stats=pd.read_csv("reference/ref_tristate2.csv"),  # ref is a mean
)

In [6]:
def individual_generator(id_=-1, cb_init=None):
    return Solution(GRNMain(5, 0, 1, generate_funcs=cb_init), id_=id_)

In [7]:
class Solution:
    def __init__(self, grn, id_=0, parent=-1):
        self.id = id_
        self.grn = grn
        self.parent = parent
        self.fit = -1
        self.stats = None
        
    def copy(self, id_=0):
        return Solution(self.grn.copy(), id_=id_, parent=self.id)
        
    def mutate(self):
        mutate_grn(self.grn)

In [8]:
def score_bb_size(bb, ref, *args, **kwargs):
    s_prog = score_stats_norm(bb.stats, ref.stats, col_stats=f"progenitor_pop_size",
                     col_ref="progenitor_pop_size", norm=2.0, *args, **kwargs)
    s_all = score_stats_norm(bb.stats, ref.stats, col_stats=f"whole_pop_size",
                     col_ref="whole_pop_size", norm=2.0, *args, **kwargs)
    s_neuron = score_stats_norm(bb.stats, ref.stats, col_stats=f"neuron_pop_size",
                     col_ref="neuron_pop_size", col_norm_data="progenitor_pop_size",
                       col_norm_ref="progenitor_pop_size", norm=2.0, *args, **kwargs)

    return np.mean(s_prog + s_all + s_neuron)

In [9]:
def setup_tag(cp):
    indexes = list(cp.base_population.keys())
    shuffle(indexes)
    splits = np.array_split(indexes, 3)
    for i, ls in enumerate(splits):
        for idx in ls:
            cp.base_population[idx].tag["subbrain"] = i

In [10]:
def get_bb(prun, grn):
    ccls = factories["grn4"](grn=grn)
    callbacks = dict(
        progenitor_pop_size=progenitor_number_callback,
        whole_pop_size=cell_number_callback,
        neuron_pop_size=neuron_number_callback,
    )
    bb = BrainModel(time_step=0.5, verbose=False, start_population=prun.size, max_pop_size=3e2,
            cell_cls=ccls, end_time=prun.end_time, start_time=59, silent=True, opti=True,
              run_tissue=False, monitor_callbacks=callbacks, tag_func=setup_tag)
    return bb

In [11]:
def run_grn(prun, grn):
    get_bb(prun, grn)
    bb.run()
    return bb

In [12]:
def fitness_multistep(prun, grn, steps):
    total_fitness = 0
    stop = False
    previous_time = None
    bb = get_bb(prun, grn)
    # first step
    for step in steps:
        if not bb.run_until(step.end_time):
            stop = True
        # score_step = score_both_size(bb.stats, prun.ref, max_step=step.end_time, min_step=previous_time)
        score_step = step.score_func(bb, prun.ref, max_step=step.end_time, min_step=step.start_time)
        fitness_step = 1.0 / score_step
        fitness_step = min(fitness_step, step.max_fitness)
        total_fitness += fitness_step
        if fitness_step < step.min_fitness or stop:
            return total_fitness, bb.stats
        else:
            previous_time = step.end_time
            step.passed()
        
    return total_fitness, bb.stats

In [13]:
def mean_sd_fitness(prun, grn, run=3):
    fitnesses = [fitness_multistep(prun, grn) for i in range(run)]
    return np.mean(fitnesses), np.std(fitnesses)

In [14]:
def multi_fitness(*args):
    fitnesses = [fitness_multistep(*args) for i in range(3)]
    scores = [x[0] for x in fitnesses]
    return fitnesses[scores.index(min(scores))]

In [15]:
def do_init(prun):
    return individual_generator(provide_id(), prun.cb_init)

def do_fitness(prun, sol):
    fitness, stats = fitness_multistep(prun, sol.grn, prun.steps)
    return fitness, stats

def do_selection(prun, pop_fit, pop):
    if len(pop) < prun.min_pop:
        return individual_generator(provide_id(), prun.cb_init)
    
    if prun.normalize:
        pop_fit = normalize_fitness_values(pop_fit)
    
    pop_fit = [p**prun.exponent for p in pop_fit]
    
    return weighted_selection_one(pop, pop_fit, lambda x: individual_generator(x, prun.cb_init), new_fitness=0.5, id_=provide_id())[0]

def do_mutation(prun, sol):
    sol.mutate()
    return sol

In [16]:
class ObjectiveStep(O):
    start_time = 0
    end_time = 0
    max_fitness = 3
    min_fitness = 1
    name = ""
    _passed = False
    
    def reset(self):
        self._passed = False
    
    def passed(self):
        if self._passed:
            return
        print(f"Step {self.name} passed !")
        self._passed = True
    
example_steps = [
    # ObjectiveStep(name="1", start_time=50, end_time=53, score_func=score_bb_size, min_fitness=0.2),
    # ObjectiveStep(name="2", start_time=53, end_time=56, score_func=score_bb_size, min_fitness=0.2),
    # ObjectiveStep(name="3", start_time=56, end_time=59, score_func=score_bb_size, min_fitness=0.2),
    ObjectiveStep(name="4", start_time=59, end_time=62, score_func=score_bb_size, min_fitness=0.2),
    ObjectiveStep(name="5", start_time=62, end_time=65, score_func=score_bb_size, min_fitness=0.2),
    ObjectiveStep(name="6", start_time=65, end_time=68, score_func=score_bb_size, min_fitness=0.2),
    ObjectiveStep(name="7", start_time=68, end_time=71, score_func=score_bb_size, min_fitness=0.2),
    ObjectiveStep(name="8", start_time=71, end_time=74, score_func=score_bb_size, min_fitness=0.2),
    ObjectiveStep(name="9", start_time=74, end_time=77, score_func=score_bb_size, min_fitness=0.2),
    ObjectiveStep(name="10", start_time=77, end_time=80, score_func=score_bb_size, min_fitness=0.2),
    ObjectiveStep(name="11", start_time=80, end_time=83, score_func=score_bb_size, min_fitness=0.2),
    ObjectiveStep(name="12", start_time=83, end_time=86, score_func=score_bb_size, min_fitness=0.2),
]

class ParamRun(O):
    pop_size = 10
    n_gen = 20
    current_gen = 0
    end_time = 86
    ref = REF
    min_pop = 20
    max_pop = 50

def get_prun(size=5, exponent=1):
    prun = ParamRun()
    prun.cb_init = dict()
    prun.size = size
    prun.exponent = exponent
    prun.steps = example_steps
    return prun

In [17]:
def main(prun):
    prun.history = defaultdict(dict)
    # exporter = Exporter()
    best = 0
    sol = do_init(prun)
    pop = [sol]
    for generation in range(prun.n_gen * prun.pop_size):
        # args.generation = generation
        # objective.new_trial()
        fit, stats = do_fitness(prun, sol)
        sol.fit = fit
        sol.stats = stats
        
        # history
        # print(f"Fitness = {fit}", end="\t\t")
        if generation % 100 == 0:
            print(f"Step {generation}")
        if fit > best:
            print(f"++ Best {fit} for generation {generation}")
            best = fit
            
        monitor = sol
        prun.history[generation] = monitor
        # exporter(monitor, f"generation_g{generation}")
        
        # TODO get the stats associated with the best scores
        sub_pop = pop[-prun.max_pop:]
        sol = do_selection(prun, [s.fit for s in sub_pop], sub_pop)
            
        sol = do_mutation(prun, sol)
        pop.append(sol)
        
    print("OVER")
    return best

In [18]:
prun = get_prun()
# res = main(prun)

In [19]:
def get_score_parametrized(cb_init, exponent, normalize, name=None):
    args = get_prun(exponent=exponent)
    args.cb_init = cb_init
    args.name = name
    args.normalize = normalize
    main(args)
    p1 = L(args.history.values()).fit
    idx = p1.index(max(p1))
    sol = args.history[idx]
    scores = [fitness_multistep(args, sol.grn, args.steps)[0] for i in range(10)]
    print("Final score", np.mean(scores), "+-", np.std(scores))
    return np.mean(scores), np.std(scores), max(p1)

In [28]:
model = sm.StringModel("expfit_exp{exponent}_n{normalize}_i{i}")
gen = product(
    [1, 2, 3],
    [True, False]
)

In [21]:
callback_init = dict(
    init=lambda: np.random.beta(1.5, 3) * 3,
    b=lambda: np.random.beta(1.5, 3) * 10,
    expr=lambda: np.random.beta(1.5, 3) * 3,
    deg=lambda: np.random.beta(1.5, 3) * 1,
)

In [29]:
# main loop
exporter = Exporter(name="exp_callbacks3_210222")
for (exponent, normalize), i in product(gen, range(10)):
    name = model.fill(exponent=exponent,
                      normalize=normalize,
                      i=i)
    print(name)
    if name in exporter.list():
        continue
    res = get_score_parametrized(callback_init, exponent, normalize)
    exporter(res, name)

Exporting at output/exp_callbacks3_210222
expfit_exp1_nTrue_i0
Step 4 passed !
Step 5 passed !
Step 6 passed !
Step 7 passed !
Step 8 passed !
Step 9 passed !
Step 10 passed !
Step 11 passed !
Step 0
++ Best 2.4105419919027464 for generation 0
Step 12 passed !
++ Best 2.4519315444155003 for generation 36
++ Best 2.7095173014284137 for generation 38
++ Best 2.7785726718266397 for generation 53
++ Best 3.82936759056421 for generation 54
++ Best 3.9996528113658356 for generation 64
Step 100
++ Best 4.086460485812541 for generation 114
OVER
Final score 2.426988967976802 +- 0.41660445049011013
expfit_exp1_nTrue_i1
Step 0
++ Best 0.1657393384116567 for generation 0
++ Best 0.16618320854305166 for generation 3
++ Best 0.36332177283239575 for generation 4
++ Best 0.39773281498281254 for generation 9
++ Best 0.731845530336097 for generation 13
++ Best 1.0697755212366555 for generation 28
++ Best 1.3083191213541374 for generation 41
++ Best 1.4798167928113517 for generation 51
++ Best 1.53117160

In [ ]:
show_curve(sol.stats, REF.stats)

In [30]:
dicts = [dict(**model.extract(x), **dict(zip(("mean", "sd", "max"), exporter.load(x)))) for x in exporter.list() if model.match(x)]

In [31]:
df = pd.DataFrame(dicts)

In [33]:
df.head()

,exponent,normalize,i,mean,sd,max
0,3,False,6,1.512011,0.335059,2.178653
1,1,True,4,0.529689,0.362774,0.701667
2,1,True,2,0.663373,0.246240,1.279343
3,2,False,6,0.442060,0.143916,0.682825
4,3,False,1,2.404735,1.340234,4.218104


In [35]:
df.groupby(["exponent", "normalize"]).mean()

mean        sd       max
exponent normalize                              
1        False      1.031787  0.418310  1.775090
         True       1.454590  0.752649  2.649117
2        False      1.012074  0.496301  2.213516
         True       0.928808  0.609160  2.630302
3        False      1.305441  0.691673  2.641665
         True       1.212822  0.649587  2.687256